# Dynamic Travel Advisor

In [ ]:
from crewai import Agent, Task, LLM, Crew
from pydantic import BaseModel, Field
from typing import List
import os
from crewai.flow.flow import Flow, start, listen, router

## Use gpt-4

In [ ]:
# Set up API key
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"
llm = LLM(model="gpt-4")

## Agents

In [6]:
# Travel Planner Agent
planner_agent = Agent(
    llm=llm,
    role="Travel Planner",
    backstory="An expert in planning group vacations, considering budgets, destinations, and activities.",
    goal="Create a vacation plan based on traveler names, departure city, and destination.",
    verbose=True,
)

# Travel Validator Agent
validator_agent = Agent(
    llm=llm,
    role="Travel Plan Validator",
    backstory="An experienced travel advisor who ensures all plans meet safety, budget, and feasibility constraints.",
    goal="Evaluate vacation plans and validate if they meet predefined constraints.",
    verbose=True,
)

## Tasks

In [7]:
def create_vacation_plan_task(names, city, destination) -> Task:
    return Task(
        description=(
            f"Create a detailed vacation plan for the following travelers: {', '.join(names)}, "
            f"departing from {city} to {destination}. Ensure the plan includes "
            f"activities, accommodation, and estimated costs."
        ),
        expected_output="A detailed vacation plan including activities, accommodation, and costs.",
        agent=planner_agent,
    )


def validate_vacation_plan_task(vacation_plan) -> Task:
    return Task(
        description=(
            f"Evaluate the following vacation plan: {vacation_plan}. "
            f"Ensure it meets the following constraints: budget-friendly, includes at least two activities, "
            f"and has clear accommodation details. Reply with 'Valid' if it meets the constraints, "
            f"otherwise reply with 'Invalid'."
        ),
        expected_output="Valid or Invalid.",
        agent=validator_agent,
    )

## Flow

In [8]:
class TravelState(BaseModel):
    vacation_plan: str = ""
    is_plan_valid: bool = False
    generation_attempts_left: int = 2

In [9]:
class TravelAdvisorFlow(Flow[TravelState]):
    names: List[str] = Field(description="List of travelers")
    city: str = Field(description="Departure city")
    destination: str = Field(description="Dest city")

    def __init__(self, names: List[str], city: str, destination: str):
        super().__init__()
        self.names = names
        self.city = city
        self.destination = destination

    @start()
    def generate_vacation_plan(self):
        print("Generating vacation plan")
        task = create_vacation_plan_task(self.names, self.city, self.destination)
        crew = Crew(agents=[planner_agent], tasks=[task])
        result = crew.kickoff()
        self.state.vacation_plan = result.raw
        print("Vacation plan generated!")

    @listen(generate_vacation_plan)
    def validate_vacation_plan(self):
        print("Start validation of the plan...")
        task = validate_vacation_plan_task(self.state.vacation_plan)
        crew = Crew(agents=[validator_agent], tasks=[task])
        result = crew.kickoff()
        self.state.is_plan_valid = "Valid" in result.raw
        print("Validation complete", "Valid" if self.state.is_plan_valid else "Invalid")

    @router(validate_vacation_plan)
    def route_vacation_plan(self):
        if self.state.is_plan_valid:
            return "valid"
        elif self.state.generation_attempts_left == 0:
            return "not_feasible"
        else:
            return "regenerate"

    @listen("valid")
    def finalize_vacation_plan(self):
        with open("vacation_plan.txt", "w") as file:
            file.write(self.state.vacation_plan)
        print("Vacation plan saved in file")

    @listen("regenerate")
    def regenerate_vacation_plan(self):
        self.state.generation_attempts_left -= 1
        self.generate_vacation_plan()

    @listen("not_feasible")
    def notify_user(self):
        print("Plan is not feasible, I'm sorry :(")


## Main

Note: `nest_asyncio` allows `asyncio.run` to work in Jupyter Notebooks by enabling nested event loops.

In [20]:
import nest_asyncio
nest_asyncio.apply()

def kickoff():
    names = ["Alice", "Robert", "Charlie"]
    city = "Berlin"
    destination = "Rome"

    travel_flow = TravelAdvisorFlow(names, city, destination)

    travel_flow.kickoff()

kickoff()

Generating vacation plan
# Agent: Travel Planner
## Task: Create a detailed vacation plan for the following travelers: Alice, Robert, Charlie, departing from Berlin to Rome. Ensure the plan includes activities, accommodation, and estimated costs.


Overriding of current TracerProvider is not allowed




# Agent: Travel Planner
## Final Answer: 
Day 1: Departure and Arrival

- Depart from Berlin via Lufthansa, flight cost approx. €150 per person.
- Arrive in Rome, after the airport transfer, check into Hotel Artemide for €200 per night for a triple occupancy room.
- Have a welcome dinner at La Piazzetta restaurant, estimated cost €25 per person.

Day 2: Exploring Rome

- Breakfast at the hotel, included in the room price.
- Take a tour of the Colosseum ($25 per person) and Roman Forum.
- Lunch at Trattoria da Valentino, estimated cost €30 per person.
- Visit the Pantheon and Trevi Fountain, free admission.
- Dinner at Ristorante Il Gabriello, estimated cost €40 per person.

Day 3: Vatican City

- Breakfast at the hotel.
- Visit Vatican City, entrance to the Vatican Museums and Sistine Chapel is €26 per person.
- Lunch at Pizzarium Bonci, estimated cost €20 per person.
- Visit St. Peter's Basilica, free entry. 
- Have dinner at Osteria delle Commari, estimated cost €35 per person.

Da